In [ ]:
import random
import string
import os

In [ ]:
def generate_random_files(byte_size):
    random_text = ''.join(random.choices(string.ascii_letters + string.digits + string.punctuation + ' ', k=byte_size))
    return random_text
def write_to_file(directory,filename, byte_size):
    
    file_path=os.path.join(directory,filename)

    random_text=generate_random_files(byte_size)

    with open(file_path,'w') as file:
        file.write(random_text)

    print(f'File "{filename}" with size {byte_size} bytes has been generated.')

sizes=[8, 64, 512, 4096, 32768, 262144, 2097152]

directory='random_files'

for size in sizes:
    filename = f'random_text_{size}.txt'
    write_to_file(directory,filename,size)

